In [1]:
from embeddings import EmbeddingsResponder
from entity_extraction import Extractor
from factual import FactualResponder
from data_repository import DataRepository
from intent_classifier import IntentClassifier, EmbeddingBasedIntentClassifier, MLPBasedIntentClassifier
from recommender import RecommendationResponder

/opt/miniconda3/envs/atai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_repository = DataRepository()

========== Initializing Data Repository ==========
========== Loading NER Embeddings ==========
========== Loading graph ==========
========== Loading data for factual QA ==========
========== Loading data for embeddings ==========
========== Loading External Data ==========
========== Data Repository initialized ==========


In [3]:
intent_classifier_emb = EmbeddingBasedIntentClassifier(data_repository)
intent_classifier_mlp = MLPBasedIntentClassifier(data_repository)
extractor = Extractor(data_repository)
embeddings = EmbeddingsResponder(data_repository, extractor, intent_classifier=intent_classifier_mlp, emb_intent_classifier=intent_classifier_emb)
factual = FactualResponder(data_repository, extractor, emb_intent_classifier=intent_classifier_emb, mlp_intent_classifier=intent_classifier_mlp)
recommender = RecommendationResponder(data_repository, extractor, mlp_intent_classifier=intent_classifier_mlp)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
recommender.answer_query("Recommend movies like Baby's Day Out and Home Alone")

["Baby's Day Out", 'Home Alone', 'Home Alone 2: Lost in New York', 'Home Alone 3', 'A Magician Home Alone', 'Planes, Trains and Automobiles', 'Only the Lonely', 'Dennis the Menace', 'Bedtime Stories', 'Uncle Buck', 'The Nightmare Before Christmas', 'Christmas with the Kranks', 'Addams Family Values', 'Relative Strangers', 'Adventures in Babysitting', 'Steal Big Steal Little', 'Jingle All the Way', 'The Goonies', "National Lampoon's European Vacation", 'Agent Cody Banks 2: Destination London']
['Home Alone 2: Lost in New York', 'Home Alone 3', 'A Magician Home Alone', 'Planes, Trains and Automobiles', 'Only the Lonely', 'Dennis the Menace', 'Bedtime Stories', 'Uncle Buck', 'The Nightmare Before Christmas', 'Christmas with the Kranks', 'Addams Family Values', 'Relative Strangers', 'Adventures in Babysitting', 'Steal Big Steal Little', 'Jingle All the Way', 'The Goonies', "National Lampoon's European Vacation", 'Agent Cody Banks 2: Destination London']


['Home Alone 2: Lost in New York', 'Home Alone 3', 'A Magician Home Alone']

In [5]:
recommender.answer_query("I liked Robert Downey Jr in The Avengers. Can you recommend me some movies like that?")

['The Avengers', 'Thor: The Dark World', 'Marvel Cinematic Universe Phase One', 'Avengers: Age of Ultron', 'Avengers: Endgame', 'Captain America: Civil War', 'Iron Man 2', 'The Infinity Saga', 'Thor', 'Avengers: Infinity War', 'Avengers: Age of Ultron final poster', 'Iron Man', 'Doctor Strange', 'Captain America: The Winter Soldier', 'Iron Man 3', 'Spider-Man: Homecoming', 'Ant-Man', 'Captain America: The First Avenger', 'Star Trek Into Darkness', 'John Carter']
['Thor: The Dark World', 'Marvel Cinematic Universe Phase One', 'Avengers: Age of Ultron', 'Avengers: Endgame', 'Captain America: Civil War', 'Iron Man 2', 'The Infinity Saga', 'Thor', 'Avengers: Infinity War', 'Avengers: Age of Ultron final poster', 'Iron Man', 'Doctor Strange', 'Captain America: The Winter Soldier', 'Iron Man 3', 'Spider-Man: Homecoming', 'Ant-Man', 'Captain America: The First Avenger', 'Star Trek Into Darkness', 'John Carter']


['Thor: The Dark World',
 'Marvel Cinematic Universe Phase One',
 'Avengers: Age of Ultron']

In [6]:
recommender.answer_query("Suggest some Horror movies for me to watch.")

['Dracula: Dead and Loving It (1995)',
 'Copycat (1995)',
 'From Dusk Till Dawn (1996)']

In [ ]:
import pandas as pd

movies_df = pd.read_csv("data/movies.csv")
ratings_df = pd.read_csv("data/ratings.csv")

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split("|"))

In [ ]:
movies_df

In [ ]:
# get horror films
horror_movies = movies_df[movies_df['genres'].apply(lambda x: 'Horror' in x and 'Sci-Fi' in x)]

In [ ]:
horror_movies[:10]

In [ ]:
ratings_df

In [ ]:
recommender.answer_query("I liked Robert De Niro in Taxi Driver, can you recommend me something similar?")

In [ ]:
extractor.extract_ner("I liked Robert De Niro in Taxi Driver, can you recommend me something similar?")

In [ ]:
movies_df[movies_df['title'].str.contains("Star Wars")]

In [ ]:
mov1 = "Baby's Day Out"
mov2 = "Home Alone"

# get the movies where the genres overlap between the two movies
intersecting_genre1 = movies_df[movies_df['title'].str.contains(mov1)]['genres']
intersecting_genre2 = movies_df[movies_df['title'].str.contains(mov2)]['genres']
combined= set(intersecting_genre1.values[0]).intersection(set(intersecting_genre2.values[0]))

In [ ]:
combined

In [ ]:
list_of_all_genres = []
for i in range(len(movies_df)):
    list_of_all_genres.extend(movies_df['genres'].values[i])

list_of_all_genres = list(set(list_of_all_genres))

In [ ]:
list_of_all_genres

In [ ]:
from fuzzywuzzy import process

process.extractOne("Give me recommendations of a  film", list_of_all_genres)[0]


In [ ]:
extractor.extract_ner("Give me recommendations of a horror film")